# Experimento 7
***
- Conjunto de Dados: VinBigData
- Testando a predição por multi-classe
- Arquitetura utilizada: Inception

### Importação dos pacotes

In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint

import warnings
warnings.filterwarnings("ignore")

### Pré-processamento nos dados

In [2]:
# lendo os dados de um arquivo csv
dataframe = pd.read_csv('/content/drive/MyDrive/vinbigdata/train.csv')
# criando uma coluna com os caminhos relativos as imagens
dataframe['image_path'] = '/content/drive/MyDrive/vinbigdata/train/' + dataframe.image_id + '.jpg'

In [3]:
print('total de imagens disponíveis:', str(len(set(dataframe['image_path']))))

total de imagens disponíveis: 15000


In [4]:
# visualizando os casos disponíveis
dataframe['class_name'].value_counts()

No finding            31818
Aortic enlargement     7162
Cardiomegaly           5427
Pleural thickening     4842
Pulmonary fibrosis     4655
Nodule/Mass            2580
Lung Opacity           2483
Pleural effusion       2476
Other lesion           2203
Infiltration           1247
ILD                    1000
Calcification           960
Consolidation           556
Atelectasis             279
Pneumothorax            226
Name: class_name, dtype: int64

In [5]:
# removendo os casos não relativos a distúrbios pulmonares
dataframe = dataframe[dataframe.class_name != 'Aortic enlargement']
dataframe = dataframe[dataframe.class_name != 'Cardiomegaly']
dataframe = dataframe[dataframe.class_name != 'Other lesion']

In [6]:
# visualizando os casos disponíveis
dataframe['class_name'].value_counts()

No finding            31818
Pleural thickening     4842
Pulmonary fibrosis     4655
Nodule/Mass            2580
Lung Opacity           2483
Pleural effusion       2476
Infiltration           1247
ILD                    1000
Calcification           960
Consolidation           556
Atelectasis             279
Pneumothorax            226
Name: class_name, dtype: int64

In [7]:
# separando os casos rotulados como normais e anormais
normal_cases = dataframe[(dataframe.class_id == 14) & (dataframe.class_name == 'No finding')]
abnormal_cases = dataframe[(dataframe.class_id != 14) & (dataframe.class_name != 'No finding')]

print('total de dados após a filtração:', str(len(set(normal_cases['image_path'])) + len(set(abnormal_cases['image_path']))))

total de dados após a filtração: 13952


In [8]:
# removendo as imagens repetidas
normal_data = normal_cases[['image_path', 'class_name']].drop_duplicates(subset = 'image_path', )
abnormal_data = abnormal_cases[['image_path', 'class_name']].drop_duplicates(subset = 'image_path', )

# criando dataframes especifos com caminhos para as imagens e rótulos
normal_data['target'] = 'normal'
abnormal_data['target'] = 'abnormal'

In [9]:
# visualizando a quantidade de exemplos após a remoção de duplicatas
abnormal_data['class_name'].value_counts()

Pleural thickening    901
Pulmonary fibrosis    742
Lung Opacity          427
Nodule/Mass           339
Pleural effusion      328
Calcification         167
Infiltration          163
ILD                   152
Consolidation          59
Atelectasis            41
Pneumothorax           27
Name: class_name, dtype: int64

In [10]:
print('quantidade de dados rotulados como normais:', len(normal_data))
print('quantidade de dados rotulados como anormais:', len(abnormal_data))

quantidade de dados rotulados como normais: 10606
quantidade de dados rotulados como anormais: 3346


In [11]:
# removendo 69% dos casos normais para balancear os dados
normal, _ = train_test_split(normal_data, test_size = 0.90, random_state = 42)

In [12]:
print('quantidade de dados rotulados como normais:', len(normal))
print('quantidade de dados rotulados como anormais:', len(abnormal_data))

quantidade de dados rotulados como normais: 1060
quantidade de dados rotulados como anormais: 3346


In [13]:
# concatenando os dataframes de casos normais e anormais
full_data = pd.concat([normal, abnormal_data])

In [14]:
# misturando todos os dados do dataframe e reiniciando os valores dos índices 
full_data = full_data.sample(frac = 1, axis = 0, random_state = 42).reset_index(drop=True)

In [15]:
# separando os dados de treinamento e de teste
train_df, test_df = train_test_split(full_data, stratify = full_data['target'],
                                     test_size = 0.2, random_state = 42)

In [16]:
# separando os dados de validação dos dados de treinamento
train_df, validation_df = train_test_split(train_df, stratify = train_df['target'],
                                           test_size = 0.2, random_state = 42)

In [17]:
# visualizando a quantidade de dados
print('quantidade de imagens de treinamento:', len(train_df['image_path']))
print('quantidade de rótulos de treinamento:', len(train_df['class_name']))
print('quantidade de imagens de teste:', len(test_df['image_path']))
print('quantidade de rótulos de teste:', len(test_df['class_name']))
print('quantidade de imagens de validação:', len(validation_df['image_path']))
print('quantidade de rótulos de validação:', len(validation_df['class_name']))

quantidade de imagens de treinamento: 2819
quantidade de rótulos de treinamento: 2819
quantidade de imagens de teste: 882
quantidade de rótulos de teste: 882
quantidade de imagens de validação: 705
quantidade de rótulos de validação: 705


### Aplicando mudança de escala típica

In [18]:
# normalizando as imagens de treinamento e aplicando aumento de dados
image_generator = ImageDataGenerator(rescale = 1./255.,
                                     rotation_range = 10, zoom_range = 0.2)

# criando o gerador de imagens de treinamento 
train_generator = image_generator.flow_from_dataframe(
                                                      dataframe = train_df,
                                                      directory = '',
                                                      x_col = 'image_path',
                                                      y_col = 'class_name',
                                                      batch_size = 32,
                                                      seed = 42,
                                                      shuffle = True,
                                                      class_mode = 'categorical',
                                                      target_size = (256, 256))

# normalizando as imagens de teste e validação
test_datagen = ImageDataGenerator(rescale = 1./255.)

# criando o gerador de imagens de validação 
valid_generator = test_datagen.flow_from_dataframe(
                                                      dataframe = validation_df,
                                                      directory = '.', 
                                                      x_col = 'image_path',
                                                      y_col = 'class_name',
                                                      batch_size = 32,
                                                      seed = 42,
                                                      shuffle = True,
                                                      class_mode = 'categorical',
                                                      target_size = (256, 256))

test_generator = test_datagen.flow_from_dataframe(
                                                  dataframe = test_df, 
                                                  directory = '.',
                                                  x_col = 'image_path',
                                                  y_col = 'class_name',
                                                  batch_size = 32,
                                                  seed = 42,
                                                  shuffle = True,
                                                  class_mode = 'categorical',
                                                  target_size = (256, 256))

Found 2819 validated image filenames belonging to 12 classes.
Found 705 validated image filenames belonging to 12 classes.
Found 882 validated image filenames belonging to 12 classes.


In [19]:
# visualizando a ordem númerica das classes
train_generator.class_indices

{'Atelectasis': 0,
 'Calcification': 1,
 'Consolidation': 2,
 'ILD': 3,
 'Infiltration': 4,
 'Lung Opacity': 5,
 'No finding': 6,
 'Nodule/Mass': 7,
 'Pleural effusion': 8,
 'Pleural thickening': 9,
 'Pneumothorax': 10,
 'Pulmonary fibrosis': 11}

### Preparando a rede neural convolucional

In [20]:
# baixando os pesos treinados da rede inception
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2021-04-16 00:29:43--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.203.128, 74.125.20.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M  73.9MB/s    in 1.1s    

2021-04-16 00:29:45 (73.9 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [21]:
# referenciando o local em que os pesos estão armazenados
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

# carregando a arquitetura inception pré-treinada
pre_trained_model = InceptionV3(input_shape = (256, 256, 3), 
                                include_top = False, 
                                weights = None)

# carregando os pesos treinados com outros dados 
pre_trained_model.load_weights(local_weights_file)

# obtendo a última camada como sendo a nomeada por 'mixed7'
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

In [22]:
x = layers.GlobalAveragePooling2D()(last_output)
# adicionando uma camada densa com 512 neurônios
x = layers.Dense(units = 512, activation = tf.nn.relu)(x)     
# conecatando a rede uma camada com 128 neurônios e função de ativação relu
x = layers.Dense(units = 256, activation = tf.nn.relu)(x) 
# aplicando uma camada de dropout com uma taxa de 20% (normalização)
x = layers.Dropout(rate = 0.2)(x)      
# adicionando uma camada de saída com um neurônio e uma função de ativação sigmoide
x = layers.Dense  (units = 12, activation = tf.nn.softmax)(x)    

# conecatando as camadas definidas acima com a arquitetura inception
model = Model(pre_trained_model.input, x) 

# compilando a rede 
model.compile(optimizer = optimizers.RMSprop(learning_rate = 0.0001), loss = 'categorical_crossentropy', 
              metrics = ['acc'])

In [23]:
# definindo as flags iniciais  
pre_trained_model.trainable = True
set_trainable = False

# para a arquitetura inception, a rede será retreinada a partir da camada 'mixed6'
for layer in pre_trained_model.layers:
    if layer.name == 'mixed6':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [24]:
# definindo o caminho pelo qual os pesos serão armazenados 
filepath = "transferlearning_weights.hdf5"
# callback para salvar o melhor valor dos pesos em relação ao desempenho com os dados de validação 
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')

In [25]:
# definindo um array de callbacks
callbacks = [checkpoint]

In [26]:
# treinando a rede neural convolucional
history = model.fit_generator(train_generator, steps_per_epoch = 2819 // 32, 
                              validation_data = valid_generator, validation_steps = 705 // 32,
                              callbacks = callbacks, epochs = 50, use_multiprocessing = True,
                              workers = 8)

Epoch 1/50
88/88 [==============================] - 335s 4s/step - loss: 2.1778 - acc: 0.2604 - val_loss: 1.9109 - val_acc: 0.3793

Epoch 00001: val_acc improved from -inf to 0.37926, saving model to transferlearning_weights.hdf5
Epoch 2/50
88/88 [==============================] - 88s 930ms/step - loss: 1.7567 - acc: 0.3953 - val_loss: 1.8912 - val_acc: 0.4020

Epoch 00002: val_acc improved from 0.37926 to 0.40199, saving model to transferlearning_weights.hdf5
Epoch 3/50
88/88 [==============================] - 89s 940ms/step - loss: 1.6579 - acc: 0.4206 - val_loss: 2.0443 - val_acc: 0.3764

Epoch 00003: val_acc did not improve from 0.40199
Epoch 4/50
88/88 [==============================] - 89s 937ms/step - loss: 1.6101 - acc: 0.4309 - val_loss: 1.8905 - val_acc: 0.4304

Epoch 00004: val_acc improved from 0.40199 to 0.43040, saving model to transferlearning_weights.hdf5
Epoch 5/50
88/88 [==============================] - 89s 941ms/step - loss: 1.4526 - acc: 0.4979 - val_loss: 1.7241 -

### Salvando o modelo

In [27]:
# carregando o melhor peso obtido para o modelo
best_model = model
best_model.load_weights('/content/transferlearning_weights.hdf5')

In [28]:
# salvando os dois modelos obtidos durante o treinamento
model.save('model1')
best_model.save('model2')

INFO:tensorflow:Assets written to: model1/assets
INFO:tensorflow:Assets written to: model2/assets


In [31]:
# testando a capacidade de generalização do modelo
model.evaluate(test_generator)

28/28 [==============================] - 793s 29s/step - loss: 3.7552 - acc: 0.4127


[3.755213975906372, 0.4126984179019928]

### Métricas de avaliação do modelo

In [32]:
# carregando os dados de teste
for i in range(0, 42):
  (x1, y1) = test_generator[i]
  if i == 0:
    x, y = x1, y1
  else:
    x = np.concatenate((x, x1))
    y = np.concatenate((y, y1))

ValueError: ignored

In [ ]:
# realizando a predição para os dados de teste
predict = model.predict(x)

In [ ]:
predict[1]

In [ ]:
for i in predict:
  print(i)
  break

In [ ]:
a = thresholds(0.5,predict)
a[0]

In [ ]:
a = np.array(a)
a = a.reshape((1327, 12))
a.shape

In [ ]:
y[0]

In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix

def thresholds(limiar, predict):
  '''predição para diferentes thresholds'''

  aux = list()
  for value in predict:
    for i in range(0, len(value)):
      if value[i] > limiar:
        aux.append(1)
      else:
        aux.append(0)
  
  aux = np.array(aux)
  aux = aux.reshape((len(predict), len(predict[0]))).astype('float32')
  
  return aux

def precision_recall_accuracy_curve(predict, y):
  ''' Relaciona a curva da Precisão, Sensibilidade e Acurácia em relação a diferentes Thresholds'''

  limiares = np.arange(0, 1, 0.05)
  predicts = []
  precisions = []
  recalls = []
  accuracy = []
  flag = 0
  for i in limiares:
    predicts.append(thresholds(i, predict))
    precisions.append(precision_score(predicts[flag], y))
    recalls.append(recall_score(predicts[flag], y))
    accuracy.append(accuracy_score(predicts[flag], y))
    flag += 1
  
  return precisions, recalls, accuracy

def plot_precision_recall_accuracy_curve(precisions, recalls, accuracy):
  '''Plotando a curva de Precisão, Sensibilidade e Acurácia'''

  plt.figure(figsize = (10,5))
  plt.plot(np.arange(0, 1, 0.05), precisions, label = 'Precision')
  plt.plot(np.arange(0, 1, 0.05), recalls, label = 'Recall')
  plt.plot(np.arange(0, 1, 0.05), accuracy, label = 'Accuracy')
  plt.title('Precisão, Sensibilidade e Acurácia para diferentes Thresholds')
  plt.xlabel('Thresholds')
  plt.legend()
  plt.savefig('curve-analysis')

  return None

def best_metrics(threshold, predict, y):
  '''Melhores valores para o threshold escolhido'''

  predict_ = thresholds(threshold, predict)
  print('Matriz de Confusão:\n', confusion_matrix(np.ndarray.tolist(predict_.astype('int')),
                                                  np.ndarray.tolist(y.astype('int')), labels = ))
  print('Acurácia:', accuracy_score(predict_, y))
  print('Precisão', precision_score(predict_, y))
  print('Sensibilidade:', recall_score(predict_, y)) 
  print('F1_Score:', f1_score(predict_, y))

  return None

In [ ]:
train_generator.class_indices
labels = ['Atelectasis', 'Calcification', 'Consolidation', 'ILD', 'Infiltration', 'Lung Opacity',
          ]

In [ ]:
# plotando a curva da Precisão, Sensibilidade e Acurácia 
precisions, recalls, accuracy = precision_recall_accuracy_curve(predict[:,0], y[:,0])
plot_precision_recall_accuracy_curve(precisions, recalls, accuracy)

In [ ]:
# plotando a curva da Precisão, Sensibilidade e Acurácia 
precisions, recalls, accuracy = precision_recall_accuracy_curve(predict[:,1], y[:,1])
plot_precision_recall_accuracy_curve(precisions, recalls, accuracy)

In [ ]:
# analisando as melhores métricas encontradas para o modelo
best_metrics(threshold = 0.60, predict = predict, y = y)

In [ ]:
# analisando as melhores métricas encontradas para o modelo
best_metrics(threshold = 0.60, predict = predict[:,1], y = y[:,1])

In [ ]:
# visualizando o ganho de acurácia durante o treinamento
plt.figure(figsize = (15,5))
plt.subplot(1, 2, 1)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('model-accuracy')

# visualizando o decaimento da função de custo durante o treinamento 
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('model-loss')

### Algoritmo Grad Cam

In [ ]:
import urllib.request as url

link = 'https://raw.githubusercontent.com/Alyssonmach/class-activation-maps/main/assets/grad_cam.py'
file_ = 'grad_cam.py'
url.urlretrieve(link, file_)

from grad_cam import get_img_array, make_gradcam_heatmap, save_and_display_gradcam

In [ ]:
# tamanho padrão das imagens do modelo
img_size = (256, 256)
# importando os parâmetros de pré-processamento da rede
preprocess_input = tf.keras.applications.inception_v3.preprocess_input
# definindo a última camada da rede a ser considerada
last_conv_layer = 'mixed7'
model_builder = model
# removendo a função de ativação da última camada
model_builder.layers[-1].activation = None

In [ ]:
img, lbl = x[0], y[0]

img_path = tf.keras.preprocessing.image.array_to_img(img)

# preparando a imagem
img_array = preprocess_input(get_img_array(img_path, img_size))

# obtendo a predição do modelo
preds = model_builder.predict(img_array)
print('Classe prevista: {}'.format(lbl))

# gerando o mapa de ativação de classe (Grad-Cam)
heatmap = make_gradcam_heatmap(img_array, model_builder, last_conv_layer,
                               pred_index = 0)

# resultado final do algoritmo Grad-Cam
heatmap = save_and_display_gradcam(img_path, heatmap, cam_path = 'image1.png')

In [ ]:
img, lbl = x[5], y[5]

img_path = tf.keras.preprocessing.image.array_to_img(img)

# preparando a imagem
img_array = preprocess_input(get_img_array(img_path, img_size))

# obtendo a predição do modelo
preds = model_builder.predict(img_array)
print('Classe prevista: {}'.format(lbl))

# gerando o mapa de ativação de classe (Grad-Cam)
heatmap = make_gradcam_heatmap(img_array, model_builder, last_conv_layer,
                               pred_index = 0)

# resultado final do algoritmo Grad-Cam
heatmap = save_and_display_gradcam(img_path, heatmap, cam_path = 'image2.png')

In [ ]:
img, lbl = x[6], y[6]

img_path = tf.keras.preprocessing.image.array_to_img(img)

# preparando a imagem
img_array = preprocess_input(get_img_array(img_path, img_size))

# obtendo a predição do modelo
preds = model_builder.predict(img_array)
print('Classe prevista: {}'.format(lbl))

# gerando o mapa de ativação de classe (Grad-Cam)
heatmap = make_gradcam_heatmap(img_array, model_builder, last_conv_layer,
                               pred_index = 0)

# resultado final do algoritmo Grad-Cam
heatmap = save_and_display_gradcam(img_path, heatmap, cam_path = 'image3.png')

In [ ]:
img, lbl = x[56], y[56]

img_path = tf.keras.preprocessing.image.array_to_img(img)

# preparando a imagem
img_array = preprocess_input(get_img_array(img_path, img_size))

# obtendo a predição do modelo
preds = model_builder.predict(img_array)
print('Classe prevista: {}'.format(lbl))

# gerando o mapa de ativação de classe (Grad-Cam)
heatmap = make_gradcam_heatmap(img_array, model_builder, last_conv_layer,
                               pred_index = 0)

# resultado final do algoritmo Grad-Cam
heatmap = save_and_display_gradcam(img_path, heatmap, cam_path = 'image4.png')

### Visualizando a arquitetura do modelo

In [ ]:
# visualizando a arquitetura do modelo
model.summary()